In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import linear_model
clf = linear_model.LinearRegression()

C:\Users\student\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
path = 'C:/Users/student/Documents/卒論/'
kakei = pd.read_csv(path + 'sampledata.csv',parse_dates = [0],encoding = "Shift-JIS",engine = "python",index_col = [0])

In [3]:
X3 = kakei.drop("消費支出",axis=1)
Y3 = kakei["消費支出"]

In [3]:
from sklearn import cross_validation

C:\Users\student\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 強制投入法での重回帰（各年）

In [7]:
for i in [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]:
    j = i+2
    df = kakei["{0}-01-01".format(i):"{0}-12-01".format(j)]
    X = df.drop("消費支出",axis=1)
    Y = df["消費支出"]
    model = sm.OLS(Y,sm.add_constant(X))
    result = model.fit()
    print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   消費支出   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                     11.29
Date:                Wed, 17 Jan 2018   Prob (F-statistic):            0.00653
Time:                        15:59:58   Log-Likelihood:                -329.01
No. Observations:                  36   AIC:                             720.0
Df Residuals:                       5   BIC:                             769.1
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.102e+04   1.65e+05     -0.550      0.6

                            OLS Regression Results                            
Dep. Variable:                   消費支出   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     243.6
Date:                Wed, 17 Jan 2018   Prob (F-statistic):           3.59e-06
Time:                        15:59:58   Log-Likelihood:                -276.20
No. Observations:                  36   AIC:                             614.4
Df Residuals:                       5   BIC:                             663.5
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -2.04e+05   4.52e+04     -4.515      0.0

C:\Users\student\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1392: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - np.divide(self.nobs - self.k_constant, self.df_resid) * (1 - self.rsquared)
C:\Users\student\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1392: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - np.divide(self.nobs - self.k_constant, self.df_resid) * (1 - self.rsquared)
C:\Users\student\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1400: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
C:\Users\student\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\student\Anaconda3\lib\site-packages\scipy\stats\stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, 

### ステップワイズ法　重回帰（各年）

In [8]:
def step_aic(model, exog, endog, **kwargs):
    # exog, endogは強制的にリスト形式に変換しておく
    exog = np.r_[[exog]].flatten()
    endog = np.r_[[endog]].flatten()
    remaining = set(exog)
    selected = []  # 採用が確定された要因

    # 定数項のみのAICを計算
    formula_head = ' + '.join(endog) + ' ~ '
    formula = formula_head + '1'
    aic = model(formula=formula, **kwargs).fit().aic
#     print('AIC: {}, formula: {}'.format(round(aic, 3), formula))

    current_score, best_new_score = np.ones(2) * aic

    # 全要因を採択するか，どの要因を追加してもAICが上がらなければ終了
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:

            # 残っている要因を1つずつ追加したときのAICを計算
            formula_tail = ' + '.join(selected + [candidate])
            formula = formula_head + formula_tail
            aic = model(formula=formula, **kwargs).fit().aic
#             print('AIC: {}, formula: {}'.format(round(aic, 3), formula))

            scores_with_candidates.append((aic, candidate))

        # 最もAICが小さかった要因をbest_candidateとする
        scores_with_candidates.sort()
        scores_with_candidates.reverse()
        best_new_score, best_candidate = scores_with_candidates.pop()

        # 候補要因追加でAICが下がったならば，それを確定要因として追加する
        if best_new_score < current_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score

    formula = formula_head + ' + '.join(selected)
    print('The best formula: {}'.format(formula))
    print(current_score)
    
    return model(formula, **kwargs).fit()


In [4]:
kakei1 =kakei.rename(columns={'乾物・海藻':'乾物海藻','他の野菜・海藻加工品':'他の野菜海藻加工品','コーヒー・ココア':'コーヒーココア'},inplace=False)

In [149]:
kakei1.columns

Index(['米', 'パン', '麺類', '他の穀類', '生鮮魚介', '塩干魚介', '魚肉練製品', '他の魚介加工品', '生鮮肉',
       '加工肉', '牛乳', '乳製品', '卵', '生鮮野菜', '乾物海藻', '大豆加工品', '他の野菜海藻加工品', '生鮮果物',
       '果物加工品', '油脂', '調味料', '菓子類', '主食的調理食品', '他の調理食品', '茶類', 'コーヒーココア',
       '他の飲料', '酒類', '一般外食', '学校給食', '消費支出'],
      dtype='object')

In [121]:
for i in [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]:
    j = i+2
    df = kakei1["{0}-01-01".format(i):"{0}-12-01".format(j)]
    model = step_aic(smf.ols,['米', 'パン', '麺類', '他の穀類', '生鮮魚介', '塩干魚介', '魚肉練製品', '他の魚介加工品', '生鮮肉',
       '加工肉', '牛乳', '乳製品', '卵', '生鮮野菜', '乾物海藻', '大豆加工品', '他の野菜海藻加工品', '生鮮果物',
       '果物加工品', '油脂', '調味料', '菓子類', '主食的調理食品', '他の調理食品', '茶類', 'コーヒーココア',
       '他の飲料', '酒類', '一般外食', '学校給食'],['消費支出'],data=df)

The best formula: 消費支出 ~ 生鮮魚介 + 学校給食 + 生鮮果物 + コーヒーココア + 調味料 + 米 + 麺類 + 菓子類 + 一般外食 + 乳製品 + 他の穀類 + パン + 主食的調理食品 + 他の飲料 + 他の魚介加工品 + 牛乳 + 大豆加工品
714.096018568
The best formula: 消費支出 ~ 生鮮魚介 + 学校給食 + コーヒーココア + 生鮮果物 + 大豆加工品 + 油脂 + 生鮮肉 + 他の調理食品 + 酒類 + 茶類 + 菓子類 + パン + 加工肉
730.28310993
The best formula: 消費支出 ~ 生鮮魚介 + 学校給食 + パン + 一般外食 + 油脂 + 菓子類 + 他の調理食品 + 主食的調理食品 + 他の飲料 + 乳製品 + 米 + 生鮮野菜 + 他の穀類
694.339051299
The best formula: 消費支出 ~ 生鮮魚介 + 学校給食 + パン + 一般外食 + 菓子類 + 油脂 + 他の調理食品 + 主食的調理食品 + 酒類 + 大豆加工品 + 乳製品
717.101162893
The best formula: 消費支出 ~ 他の魚介加工品 + 学校給食 + パン + 生鮮肉 + 他の調理食品 + 果物加工品 + 調味料 + 酒類 + 他の穀類 + 生鮮魚介 + 生鮮果物 + 乳製品
719.134724346
The best formula: 消費支出 ~ 乾物海藻 + 学校給食 + コーヒーココア + 他の野菜海藻加工品 + 生鮮果物 + 他の調理食品 + 生鮮肉 + 酒類 + 茶類 + 加工肉 + 米 + 牛乳 + 油脂 + 塩干魚介 + 他の魚介加工品 + 卵 + 調味料 + 魚肉練製品 + 大豆加工品 + 他の飲料 + 乳製品 + 麺類 + 菓子類 + 果物加工品 + 主食的調理食品 + パン + 生鮮魚介 + 一般外食 + 生鮮野菜 + 他の穀類
614.395257408
The best formula: 消費支出 ~ 乾物海藻 + 学校給食 + 主食的調理食品 + 一般外食 + 他の野菜海藻加工品 + 茶類 + 他の調理食品 + 牛乳 + 魚肉練製品 + 生鮮魚介 + 果物加工品 + パン + 大豆加工品 + 麺類

In [5]:
step_kakei = kakei1.loc[:,["消費支出","生鮮魚介" , "学校給食" , "生鮮果物" , "一般外食" , "米" , "他の調理食品" , "魚肉練製品" , "主食的調理食品" , "乾物海藻" , "牛乳"]]

交差検証での性能評価

In [6]:
X2 = step_kakei.drop("消費支出",axis=1)
Y2 = step_kakei["消費支出"]

In [18]:
model = step_aic(smf.ols,['米', 'パン', '麺類', '他の穀類', '生鮮魚介', '塩干魚介', '魚肉練製品', '他の魚介加工品', '生鮮肉',
       '加工肉', '牛乳', '乳製品', '卵', '生鮮野菜', '乾物海藻', '大豆加工品', '他の野菜海藻加工品', '生鮮果物',
       '果物加工品', '油脂', '調味料', '菓子類', '主食的調理食品', '他の調理食品', '茶類', 'コーヒーココア',
       '他の飲料', '酒類', '一般外食', '学校給食'],['消費支出'],data=kakei1)

The best formula: 消費支出 ~ 生鮮魚介 + 学校給食 + 生鮮果物 + 一般外食 + 米 + 他の調理食品 + 魚肉練製品 + 主食的調理食品 + 乾物海藻 + 牛乳
2487.13570806


In [16]:
hodf = kakei["2007-01-01":"2009-12-01"] 
testdf = kakei["2010-01-01":"2010-12-01"]
X_train = hodf.drop("消費支出",axis=1)
y_train = hodf["消費支出"]
X_test = testdf.drop("消費支出",axis=1)
y_test = testdf["消費支出"]


In [15]:
model = smf.OLS(Y,smf.add_constant(X))
result = model.fit()
result.summary()

2007-01-01    296472
2007-02-01    272763
2007-03-01    313563
2007-04-01    316163
2007-05-01    293231
2007-06-01    280587
2007-07-01    291632
2007-08-01    296035
2007-09-01    281448
2007-10-01    296984
2007-11-01    282836
2007-12-01    351667
2008-01-01    309826
2008-02-01    275827
2008-03-01    312565
2008-04-01    310695
2008-05-01    288128
2008-06-01    281951
2008-07-01    298366
2008-08-01    291154
2008-09-01    281433
2008-10-01    291504
2008-11-01    284762
2008-12-01    336976
2009-01-01    291440
2009-02-01    266044
2009-03-01    310680
2009-04-01    306340
2009-05-01    285530
2009-06-01    277237
2009-07-01    285078
2009-08-01    290972
2009-09-01    277110
2009-10-01    287789
2009-11-01    284740
2009-12-01    337887
Name: 消費支出, dtype: int64

交差検証法

In [16]:
pcrdata = pd.read_csv('主成分得点データ.csv',encoding = "Shift-JIS",engine = "python")

In [3]:
y=kakei["消費支出"]

In [8]:
# step=kakei[["米","","乳製品","油脂","学校給食"]]
# pcr=pcrdata.drop("消費支出",axis=1)
step=kakei[["米","生鮮果物","主食的調理食品","一般外食","学校給食"]]
lasso=kakei[["米","生鮮果物","生鮮魚介","一般外食","学校給食","主食的調理食品","他の調理食品"]]

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
clf = linear_model.LinearRegression()
scores = cross_val_score(clf,step,y,cv=10)
print(scores)
scores.mean()

[ 0.55576086  0.21306565  0.65631449  0.71530559  0.59405033  0.60767314
  0.33977508  0.43196444  0.55542817  0.52888467]


0.51982224162906587

In [10]:
scores = cross_val_score(clf,lasso,y,cv=10)
print(scores)
scores.mean()

[ 0.92032913  0.72353318  0.91810815  0.89449508  0.90723909  0.73516071
  0.85882714  0.67960825  0.71573461  0.88121685]


0.82342521917167277